# Hugging Face Model Deployment

This notebook downloads the best performing model artifacts from MLflow and prepares them for deployment on Hugging Face Hub.

## Overview
- **Objective**: Download and prepare the best BERT model for Hugging Face deployment
- **MLflow Run ID**: ff204ab808384e77a8b1e40f56a3fd2a (BERT-base-uncased, batch size 8)
- **Model Performance**: 92.50% F1-Score, 92.33% Accuracy
- **Target Platform**: Hugging Face Hub for public model sharing


## 1. Setup and Dependencies


In [1]:
# Install required packages
%pip install mlflow huggingface_hub transformers torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import required libraries
import os
import json
import mlflow
import mlflow.pytorch
import torch
from transformers import (
    BertForSequenceClassification, 
    BertTokenizer, 
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from huggingface_hub import HfApi, Repository, create_repo
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"MLflow version: {mlflow.__version__}")


/Users/jam/serendip-travel/explainable-tourism-nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries imported successfully
PyTorch version: 2.8.0
MLflow version: 3.4.0


## 2. MLflow Configuration and Model Download


In [3]:
# MLflow configuration
# Set up MLflow tracking URI (adjust based on your setup)
try:
    # Try Databricks first
    mlflow.set_tracking_uri("databricks")
    print("Connected to Databricks MLflow")
except:
    # Fallback to local
    mlflow.set_tracking_uri("file:./mlruns")
    print("Using local MLflow tracking")

# Define the run ID for the best model
BEST_RUN_ID = "ff204ab808384e77a8b1e40f56a3fd2a"
print(f"Target MLflow Run ID: {BEST_RUN_ID}")


Connected to Databricks MLflow
Target MLflow Run ID: ff204ab808384e77a8b1e40f56a3fd2a


In [4]:
# Download model artifacts from MLflow
def download_mlflow_model(run_id, download_path="./model_artifacts"):
    """Download model artifacts from MLflow run"""
    try:
        # Create download directory
        os.makedirs(download_path, exist_ok=True)
        
        # Download the model
        model_uri = f"runs:/{run_id}/model"
        print(f"Downloading model from: {model_uri}")
        
        # Download model artifacts
        mlflow.pytorch.load_model(model_uri, dst_path=download_path)
        
        print(f"Model downloaded successfully to: {download_path}")
        return download_path
        
    except Exception as e:
        print(f"Error downloading model: {e}")
        return None

# Download the best model
model_path = download_mlflow_model(BEST_RUN_ID)


KeyboardInterrupt: 

In [ ]:
# Get run information and metrics
def get_run_info(run_id):
    """Get detailed information about the MLflow run"""
    try:
        # Get run details
        run = mlflow.get_run(run_id)
        
        print("=" * 60)
        print("MLFLOW RUN INFORMATION")
        print("=" * 60)
        print(f"Run ID: {run_id}")
        print(f"Run Name: {run.data.tags.get('mlflow.runName', 'N/A')}")
        print(f"Status: {run.info.status}")
        print(f"Start Time: {run.info.start_time}")
        print(f"End Time: {run.info.end_time}")
        
        print(f"\nParameters:")
        for key, value in run.data.params.items():
            print(f"  {key}: {value}")
            
        print(f"\nMetrics:")
        for key, value in run.data.metrics.items():
            print(f"  {key}: {value}")
            
        print(f"\nTags:")
        for key, value in run.data.tags.items():
            print(f"  {key}: {value}")
            
        return run
        
    except Exception as e:
        print(f"Error getting run info: {e}")
        return None

# Get run information
run_info = get_run_info(BEST_RUN_ID)


## 3. Model Preparation for Hugging Face


In [ ]:
# Load the downloaded model and prepare for Hugging Face
def prepare_model_for_hf(model_path, hf_model_path="./hf_model"):
    """Prepare the MLflow model for Hugging Face Hub"""
    try:
        # Create Hugging Face model directory
        os.makedirs(hf_model_path, exist_ok=True)
        
        # Load the model from MLflow
        print("Loading model from MLflow...")
        model = mlflow.pytorch.load_model(f"runs:/{BEST_RUN_ID}/model")
        
        # Load the tokenizer
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        
        # Save model and tokenizer in Hugging Face format
        print(f"Saving model to {hf_model_path}...")
        model.save_pretrained(hf_model_path)
        tokenizer.save_pretrained(hf_model_path)
        
        print("Model and tokenizer saved successfully!")
        return hf_model_path, model, tokenizer
        
    except Exception as e:
        print(f"Error preparing model: {e}")
        return None, None, None

# Prepare the model
hf_path, model, tokenizer = prepare_model_for_hf(model_path)


In [ ]:
# Create model configuration and metadata
def create_model_config(hf_path, run_info):
    """Create configuration files for Hugging Face model"""
    try:
        # Model configuration
        config = {
            "model_type": "bert",
            "architectures": ["BertForSequenceClassification"],
            "num_labels": 4,
            "id2label": {
                "0": "Regenerative & Eco-Tourism",
                "1": "Integrated Wellness", 
                "2": "Immersive Culinary",
                "3": "Off-the-Beaten-Path Adventure"
            },
            "label2id": {
                "Regenerative & Eco-Tourism": 0,
                "Integrated Wellness": 1,
                "Immersive Culinary": 2,
                "Off-the-Beaten-Path Adventure": 3
            },
            "problem_type": "multi_label_classification"
        }
        
        # Save config.json
        with open(f"{hf_path}/config.json", "w") as f:
            json.dump(config, f, indent=2)
            
        # Create README.md
        readme_content = f"""---
language: en
license: mit
tags:
- text-classification
- multi-label
- tourism
- sri-lanka
- bert
- pytorch
datasets:
- tourism-reviews-sri-lanka
metrics:
- f1
- accuracy
- precision
- recall
model-index:
- name: serendip-travel-experiential-classifier
  results:
  - task:
      type: text-classification
      name: Multi-Label Text Classification
    dataset:
      type: tourism-reviews-sri-lanka
      name: Sri Lankan Tourism Reviews
    metrics:
    - type: f1
      value: 0.9250
    - type: accuracy
      value: 0.9233
    - type: precision
      value: 0.9681
    - type: recall
      value: 0.8859
---

# Serendip Travel Experiential Classifier

A fine-tuned BERT model for classifying Sri Lankan tourist reviews into four experiential dimensions.

## Model Description

This model is a fine-tuned BERT-base-uncased model trained on Sri Lankan tourism reviews to classify text into four experiential dimensions:

1. **Regenerative & Eco-Tourism**: Travel focused on positive social and environmental impact
2. **Integrated Wellness**: Journeys combining physical and mental well-being
3. **Immersive Culinary**: Experiences centered on authentic local cuisine
4. **Off-the-Beaten-Path Adventure**: Exploration of less crowded natural landscapes

## Performance

- **F1-Score**: 92.50%
- **Accuracy**: 92.33%
- **Precision**: 96.81%
- **Recall**: 88.59%

## Usage

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("your-username/serendip-travel-classifier")
model = AutoModelForSequenceClassification.from_pretrained("your-username/serendip-travel-classifier")

# Example prediction
text = "The organic tea plantation tour was amazing! We learned about sustainable farming practices."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.sigmoid(outputs.logits)
    
# Get predicted labels
labels = ["Regenerative & Eco-Tourism", "Integrated Wellness", "Immersive Culinary", "Off-the-Beaten-Path Adventure"]
predicted_labels = [labels[i] for i, score in enumerate(predictions[0]) if score > 0.5]
print(f"Predicted labels: {predicted_labels}")
```

## Training Details

- **Base Model**: bert-base-uncased
- **Training Data**: 16,156 Sri Lankan tourism reviews
- **Learning Rate**: 2e-05
- **Batch Size**: 8
- **Epochs**: 5
- **Framework**: PyTorch with MLflow tracking

## Citation

```bibtex
@misc{{serendip-travel-classifier,
  title={{Serendip Travel Experiential Classifier}},
  author={{B M J N Balasuriya}},
  year={{2025}},
  publisher={{Hugging Face}},
  howpublished={{\\url{{https://huggingface.co/your-username/serendip-travel-classifier}}}}
}}
```
"""
        
        # Save README.md
        with open(f"{hf_path}/README.md", "w") as f:
            f.write(readme_content)
            
        print("Configuration files created successfully!")
        return True
        
    except Exception as e:
        print(f"Error creating config: {e}")
        return False

# Create configuration files
if hf_path and run_info:
    create_model_config(hf_path, run_info)


## 4. Model Testing and Validation


In [ ]:
# Test the prepared model
def test_model_prediction(model, tokenizer, test_texts):
    """Test the model with sample texts"""
    try:
        print("=" * 60)
        print("MODEL TESTING")
        print("=" * 60)
        
        labels = [
            "Regenerative & Eco-Tourism",
            "Integrated Wellness", 
            "Immersive Culinary",
            "Off-the-Beaten-Path Adventure"
        ]
        
        for i, text in enumerate(test_texts, 1):
            print(f"\nTest {i}: {text[:100]}...")
            
            # Tokenize input
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
            
            # Get predictions
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = torch.sigmoid(outputs.logits)
            
            # Display results
            print("Predicted labels:")
            for j, (label, score) in enumerate(zip(labels, predictions[0])):
                status = "✓" if score > 0.5 else "✗"
                print(f"  {status} {label}: {score:.3f}")
                
        return True
        
    except Exception as e:
        print(f"Error testing model: {e}")
        return False

# Sample test texts
test_texts = [
    "The organic tea plantation tour was amazing! We learned about sustainable farming practices and environmental conservation.",
    "The spa retreat offered incredible yoga sessions and meditation classes. Perfect for relaxation and wellness.",
    "The local cooking class was fantastic! We learned to make authentic Sri Lankan curry with fresh spices from the market.",
    "The hiking trail through the remote jungle was challenging but rewarding. We saw amazing wildlife and untouched nature."
]

# Test the model
if model and tokenizer:
    test_model_prediction(model, tokenizer, test_texts)


## 5. Hugging Face Hub Upload


In [ ]:
# Hugging Face Hub upload setup
def setup_huggingface_upload():
    """Setup Hugging Face Hub for model upload"""
    print("=" * 60)
    print("HUGGING FACE HUB SETUP")
    print("=" * 60)
    
    print("To upload your model to Hugging Face Hub, you need to:")
    print("1. Create a Hugging Face account at https://huggingface.co/")
    print("2. Generate an access token at https://huggingface.co/settings/tokens")
    print("3. Install and login to huggingface_hub:")
    print("   !pip install huggingface_hub")
    print("   !huggingface-cli login")
    print("4. Update the model name below with your username")
    
    # Model name (update with your username)
    model_name = "your-username/serendip-travel-classifier"
    print(f"\nModel will be uploaded as: {model_name}")
    
    return model_name

# Setup instructions
model_name = setup_huggingface_upload()


In [ ]:
# Upload model to Hugging Face Hub
def upload_to_huggingface(hf_path, model_name):
    """Upload the prepared model to Hugging Face Hub"""
    try:
        print("=" * 60)
        print("UPLOADING TO HUGGING FACE HUB")
        print("=" * 60)
        
        # Initialize Hugging Face API
        api = HfApi()
        
        # Create repository
        print(f"Creating repository: {model_name}")
        create_repo(model_name, exist_ok=True, private=False)
        
        # Upload all files
        print(f"Uploading files from {hf_path}...")
        api.upload_folder(
            folder_path=hf_path,
            repo_id=model_name,
            repo_type="model"
        )
        
        print(f"✅ Model successfully uploaded to: https://huggingface.co/{model_name}")
        return True
        
    except Exception as e:
        print(f"❌ Error uploading to Hugging Face: {e}")
        print("\nTroubleshooting:")
        print("1. Make sure you're logged in: huggingface-cli login")
        print("2. Check your access token permissions")
        print("3. Verify the model name is correct")
        return False

# Upload the model (uncomment when ready)
# upload_to_huggingface(hf_path, model_name)


## 6. Alternative Upload Methods


In [ ]:
# Alternative: Manual upload using git
def create_git_upload_instructions(hf_path, model_name):
    """Create instructions for manual git upload"""
    print("=" * 60)
    print("ALTERNATIVE: MANUAL GIT UPLOAD")
    print("=" * 60)
    
    print("If the API upload fails, you can use git to upload manually:")
    print()
    print("1. Clone the repository:")
    print(f"   git clone https://huggingface.co/{model_name}")
    print()
    print("2. Copy files to the repository:")
    print(f"   cp -r {hf_path}/* {model_name}/")
    print()
    print("3. Add, commit, and push:")
    print("   cd {model_name}")
    print("   git add .")
    print("   git commit -m 'Add model files'")
    print("   git push")
    print()
    print("4. Or use the web interface:")
    print(f"   https://huggingface.co/{model_name}")
    print("   - Click 'Add file' -> 'Upload files'")
    print(f"   - Upload all files from {hf_path}/")
    
    # List files to upload
    print(f"\nFiles to upload from {hf_path}:")
    if os.path.exists(hf_path):
        for file in os.listdir(hf_path):
            print(f"  - {file}")

# Create manual upload instructions
create_git_upload_instructions(hf_path, model_name)


## 7. Model Usage Examples


In [ ]:
# Create usage examples for the deployed model
def create_usage_examples():
    """Create comprehensive usage examples"""
    print("=" * 60)
    print("MODEL USAGE EXAMPLES")
    print("=" * 60)
    
    examples = """
# Example 1: Basic Usage
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("your-username/serendip-travel-classifier")
model = AutoModelForSequenceClassification.from_pretrained("your-username/serendip-travel-classifier")

# Example text
text = "The organic tea plantation tour was amazing! We learned about sustainable farming practices."

# Tokenize and predict
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.sigmoid(outputs.logits)

# Get predicted labels
labels = ["Regenerative & Eco-Tourism", "Integrated Wellness", "Immersive Culinary", "Off-the-Beaten-Path Adventure"]
predicted_labels = [labels[i] for i, score in enumerate(predictions[0]) if score > 0.5]
print(f"Predicted labels: {predicted_labels}")

# Example 2: Batch Processing
texts = [
    "The spa retreat offered incredible yoga sessions and meditation classes.",
    "The local cooking class was fantastic! We learned to make authentic Sri Lankan curry.",
    "The hiking trail through the remote jungle was challenging but rewarding."
]

# Process multiple texts
for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.sigmoid(outputs.logits)
    
    predicted_labels = [labels[i] for i, score in enumerate(predictions[0]) if score > 0.5]
    print(f"Text: {text[:50]}...")
    print(f"Labels: {predicted_labels}")
    print()

# Example 3: Confidence Scores
def get_confidence_scores(text, threshold=0.5):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.sigmoid(outputs.logits)
    
    results = []
    for i, (label, score) in enumerate(zip(labels, predictions[0])):
        results.append({
            'label': label,
            'score': float(score),
            'predicted': score > threshold
        })
    
    return results

# Get detailed confidence scores
text = "The organic tea plantation tour was amazing! We learned about sustainable farming practices."
scores = get_confidence_scores(text)
for result in scores:
    print(f"{result['label']}: {result['score']:.3f} ({'✓' if result['predicted'] else '✗'})")
"""
    
    print(examples)

# Display usage examples
create_usage_examples()


## 8. Summary and Next Steps


In [ ]:
# Final summary and next steps
def deployment_summary():
    """Provide deployment summary and next steps"""
    print("=" * 60)
    print("DEPLOYMENT SUMMARY")
    print("=" * 60)
    
    print("✅ Model Preparation Complete!")
    print(f"📁 Model files saved to: {hf_path}")
    print(f"🏷️  Model name: {model_name}")
    print(f"📊 Performance: 92.50% F1-Score, 92.33% Accuracy")
    
    print("\n📋 Next Steps:")
    print("1. 🔐 Login to Hugging Face: huggingface-cli login")
    print("2. ✏️  Update model name with your username")
    print("3. 🚀 Uncomment and run the upload function")
    print("4. 📝 Update README.md with your actual model URL")
    print("5. 🧪 Test the deployed model")
    print("6. 📢 Share your model with the community!")
    
    print("\n📁 Files Created:")
    if hf_path and os.path.exists(hf_path):
        for file in os.listdir(hf_path):
            print(f"  - {file}")
    
    print(f"\n🔗 Model will be available at:")
    print(f"   https://huggingface.co/{model_name}")
    
    print("\n🎯 Model Features:")
    print("  - Multi-label text classification")
    print("  - 4 experiential dimensions")
    print("  - BERT-base-uncased architecture")
    print("  - 92.50% F1-Score performance")
    print("  - Ready for production use")
    
    print("\n💡 Usage Tips:")
    print("  - Use threshold 0.5 for binary predictions")
    print("  - Adjust threshold based on use case")
    print("  - Consider confidence scores for uncertainty")
    print("  - Batch processing for efficiency")

# Display summary
deployment_summary()
